# Introduction

Dans ce notebook, on utilise le jeu de données issus de l'augmentation de données (ajout de taxons, API Plantnet...) afin de rechercher à nouveaux des motifs fréquents et potentielles règles d'associations intéressantes plantes-insectes.

Précédemment, nous avions recherché des motifs fréquents et règles d'associations importantes (lift supérieur à 1) plantes-insectes à partir des taxons présents dans la base du SPIPOLL : taxons _sc, _fr, _precision.

Pour un seuil de support minimal de 0.5, la recherche n'avait pas permis de mettre en évidence des motifs fréquents intéressants et règles d'associations importantes. L'abaissement du seuil à 0.4 avait permis de mettre en lumière un motif frelon-lierre, sans toutefois soulever une règle d'association importante. 

On se demande si d'autres motifs fréquents intéressants n'apparaitraient pas à un niveau de granularité plus global, grâce à l'approfondissement de la taxonomie. On va donc tester ici les niveaux de taxonomie suivant : 
- plante_famille, plante_genre, plante_espece
- insecte_ordre, insecte_super_famille, insecte_famille, insecte_sous_famille, insecte_genre, insecte_espece.

# Chargement des données

In [19]:
import pandas as pd

Décommentez la cellule suivante si vous avez besoin d'installer la librairie qui nous servira à la recherche de motifs fréquents.

In [20]:
# Installation de la librairie permettant la recherche de motifs fréquents 
#! pip install mlxtend

In [32]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [33]:
df = pd.read_csv("./spipoll.csv")

C:\Users\guilts0300\AppData\Local\Temp\ipykernel_20972\35506862.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./spipoll.csv")


In [34]:
unused_columns = ['collection_nom', 'collection_id', 'protocole_long', 'user_id',
       'photo_fleur', 'photo_plante', 'photo_feuille',               
       'plante_caractere', 'plante_inconnue',
       'photo_fleur', 'photo_plante', 'photo_feuille', 'coordonnees_GPS',
       'code_postal', 'fleur_ombre', 
       'insecte_abondance', 'insecte_commentaire', 'insecte_vu_sur_fleur',
       'nb_validation', 'nb_suggestion']

In [35]:
df.drop(columns=unused_columns, inplace=True)

In [36]:
# Transformation de la variable habitat

# One hot encoding après création de nouvelles colonnes pour chacun des habitats spécifiés entre ','
habitat_dummies = df['habitat'].str.get_dummies(',')

# Concaténation les colonnes de dummies avec le DataFrame d'origine
df = pd.concat([df, habitat_dummies], axis=1)

In [43]:
df.drop(columns=['habitat'], inplace=True)

In [37]:
df.columns

Index(['plante_famille', 'plante_genre', 'plante_espece', 'plante_sc',
       'plante_fr', 'plante_precision', 'habitat', 'grande_culture',
       'collection_date', 'collection_heure_debut', 'nebulosite',
       'temperature', 'vent', 'insecte_ordre', 'insecte_super_famille',
       'insecte_famille', 'insecte_sous_famille', 'insecte_genre',
       'insecte_espece', 'insecte_sc', 'insecte_fr',
       'insecte_denominationPlusPrecise', 'insecte_CdNomtaxref',
       'bord de l'eau', 'bord de route', 'forêt', 'grande(s) culture(s)',
       'jardin privé', 'littoral', 'parc ou jardin public', 'prairie',
       'péri-urbain', 'rochers', 'rural', 'urbain'],
      dtype='object')

# Recherche de motifs fréquents 

In [38]:
df.columns

Index(['plante_famille', 'plante_genre', 'plante_espece', 'plante_sc',
       'plante_fr', 'plante_precision', 'habitat', 'grande_culture',
       'collection_date', 'collection_heure_debut', 'nebulosite',
       'temperature', 'vent', 'insecte_ordre', 'insecte_super_famille',
       'insecte_famille', 'insecte_sous_famille', 'insecte_genre',
       'insecte_espece', 'insecte_sc', 'insecte_fr',
       'insecte_denominationPlusPrecise', 'insecte_CdNomtaxref',
       'bord de l'eau', 'bord de route', 'forêt', 'grande(s) culture(s)',
       'jardin privé', 'littoral', 'parc ou jardin public', 'prairie',
       'péri-urbain', 'rochers', 'rural', 'urbain'],
      dtype='object')

In [28]:
df['insecte_ordre'].nunique()

16

In [29]:
df['insecte_ordre'].value_counts()

Hymenoptera      227499
Diptera          212211
Coleoptera        79761
Lepidoptera       55218
Hemiptera         36776
Araneae           22221
Orthoptera         8089
Ericales           5031
Neuroptera         1099
Opiliones           509
Dermaptera          462
Mecoptera           424
Blattodea           402
Siphonophorae       242
Ephemeroptera        52
Raphidioptera        18
Name: insecte_ordre, dtype: int64

In [41]:
df['plante_famille'].nunique()

138

In [42]:
df['plante_famille'].value_counts()

Asteraceae        197080
Apiaceae          108274
Rosaceae           54316
Lamiaceae          54218
Caprifoliaceae     28031
                   ...  
Zosteraceae            2
Cytinaceae             2
Droseraceae            1
Dicksoniaceae          1
Elatinaceae            1
Name: plante_famille, Length: 138, dtype: int64

## Vérification sur un exemple courant

On teste ici sur l'ordre incluant les bourdons, abeilles, frelons, etc. On aimerait trouver des motifs fréquents avec une famille de plantes.

In [39]:
data=df.where(df['insecte_ordre']=='Hymenoptera').dropna(subset=['insecte_ordre'])

In [40]:
data.columns

Index(['plante_famille', 'plante_genre', 'plante_espece', 'plante_sc',
       'plante_fr', 'plante_precision', 'habitat', 'grande_culture',
       'collection_date', 'collection_heure_debut', 'nebulosite',
       'temperature', 'vent', 'insecte_ordre', 'insecte_super_famille',
       'insecte_famille', 'insecte_sous_famille', 'insecte_genre',
       'insecte_espece', 'insecte_sc', 'insecte_fr',
       'insecte_denominationPlusPrecise', 'insecte_CdNomtaxref',
       'bord de l'eau', 'bord de route', 'forêt', 'grande(s) culture(s)',
       'jardin privé', 'littoral', 'parc ou jardin public', 'prairie',
       'péri-urbain', 'rochers', 'rural', 'urbain'],
      dtype='object')

In [45]:
to_drop = ['plante_genre', 'plante_espece', 'plante_sc',
       'plante_fr', 'plante_precision', 'grande_culture',
       'collection_date', 'collection_heure_debut', 
        'insecte_super_famille',
       'insecte_famille', 'insecte_sous_famille', 'insecte_genre',
       'insecte_espece', 'insecte_sc', 'insecte_fr',
       'insecte_denominationPlusPrecise', 'insecte_CdNomtaxref']

In [46]:
data.drop(columns=to_drop, inplace=True)

In [50]:
data.columns

Index(['plante_famille', 'habitat', 'nebulosite', 'temperature', 'vent',
       'insecte_ordre', 'bord de l'eau', 'bord de route', 'forêt',
       'grande(s) culture(s)', 'jardin privé', 'littoral',
       'parc ou jardin public', 'prairie', 'péri-urbain', 'rochers', 'rural',
       'urbain'],
      dtype='object')

In [47]:
# One hot encoding
df_encoded = pd.get_dummies(data)
df_encoded = df_encoded.astype(bool)

In [48]:
# Recherche des motifs fréquents
frequent_itemsets = apriori(df_encoded, min_support=0.5, use_colnames=True)

# Filtrage des motifs fréquents de taille minimale égale à 2
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets = frequent_itemsets[frequent_itemsets['length'] >= 2].drop(columns='length')

# Classement par ordre décroissant de support 
frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)
frequent_itemsets

,support,itemsets
4,0.803467,"(insecte_ordre_Hymenoptera, nebulosite_0-25%)"
5,0.604517,"(insecte_ordre_Hymenoptera, temperature_20-30ºC)"
6,0.506341,"(vent_faible, irrégulier, insecte_ordre_Hymeno..."


In [49]:
frequent_itemsets['itemsets'].unique()

array([frozenset({'insecte_ordre_Hymenoptera', 'nebulosite_0-25%'}),
       frozenset({'insecte_ordre_Hymenoptera', 'temperature_20-30ºC'}),
       frozenset({'vent_faible, irrégulier', 'insecte_ordre_Hymenoptera'})],
      dtype=object)

Conclusion : Pour un seuil de support minimal de 0.5, nous n'avons pas trouvé de motifs fréquents insectes de l'ordre des hyménoptères avec une famille de plantes.
On effectue tout de même cette recherche pour les 16 ordres d'insectes (du tacon insecte_ordre).

## Sur toute la base 

In [61]:
df['insecte_ordre'].unique().tolist() 

['Hymenoptera',
 'Diptera',
 'Orthoptera',
 nan,
 'Coleoptera',
 'Hemiptera',
 'Araneae',
 'Lepidoptera',
 'Ericales',
 'Neuroptera',
 'Opiliones',
 'Siphonophorae',
 'Mecoptera',
 'Dermaptera',
 'Raphidioptera',
 'Ephemeroptera',
 'Blattodea']

In [75]:
for insect in ['Hymenoptera', 'Diptera', 'Orthoptera', 'Coleoptera', 'Hemiptera', 'Araneae', 
               'Lepidoptera', 'Ericales', 'Neuroptera', 'Opiliones', 'Siphonophorae', 
               'Mecoptera', 'Dermaptera', 'Raphidioptera', 'Ephemeroptera', 'Blattodea']:

    data=df.where(df['insecte_ordre']==insect).dropna(subset=['insecte_ordre'])
    
    #Nettoyage des colonnes inutiles (on ne garde que plante_famille pour les taxons de plantes)
    to_drop= ['plante_genre', 'plante_espece', 'plante_sc',
       'plante_fr', 'plante_precision', 'grande_culture',
       'collection_date', 'collection_heure_debut', 
        'insecte_super_famille',
       'insecte_famille', 'insecte_sous_famille', 'insecte_genre',
       'insecte_espece', 'insecte_sc', 'insecte_fr',
       'insecte_denominationPlusPrecise', 'insecte_CdNomtaxref']
    data.drop(columns=to_drop, inplace=True)
    
    # One hot encoding
    df_encoded = pd.get_dummies(data)
    df_encoded = df_encoded.astype(bool)
    
    # Algo a priori sur le jeu encodé
    frequent_itemsets = apriori(df_encoded, min_support=0.3, use_colnames=True)

    # Filtrer les motifs fréquents pour une taille minimale de 2
    frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
    frequent_itemsets = frequent_itemsets[frequent_itemsets['length'] >= 2].drop(columns='length')

    # Classement par ordre décroissant de support 
    frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)
    print(frequent_itemsets)
    print(frequent_itemsets['itemsets'].unique())
    
    # Recherche de règles d'association avec un lift supérieur à 1 
    #rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1, support_only=True)
    
    # Affichage des règles d'association intéressantes
    #if len(rules)>0: 
        #print(insect, " : ", rules)

     support                                           itemsets
10  0.803467      (insecte_ordre_Hymenoptera, nebulosite_0-25%)
13  0.604517   (insecte_ordre_Hymenoptera, temperature_20-30ºC)
14  0.506341  (vent_faible, irrégulier, insecte_ordre_Hymeno...
8   0.484297            (nebulosite_0-25%, temperature_20-30ºC)
16  0.484297  (insecte_ordre_Hymenoptera, nebulosite_0-25%, ...
7   0.456890                 (rural, insecte_ordre_Hymenoptera)
9   0.404811        (vent_faible, irrégulier, nebulosite_0-25%)
17  0.404811  (vent_faible, irrégulier, insecte_ordre_Hymeno...
6   0.373013                          (rural, nebulosite_0-25%)
15  0.373013  (insecte_ordre_Hymenoptera, rural, nebulosite_...
12  0.312480     (vent_faible, irrégulier, temperature_20-30ºC)
18  0.312480  (vent_faible, irrégulier, insecte_ordre_Hymeno...
11  0.304709   (temperature_10-20ºC, insecte_ordre_Hymenoptera)
[frozenset({'insecte_ordre_Hymenoptera', 'nebulosite_0-25%'})
 frozenset({'insecte_ordre_Hymenoptera', '

On obtient des motifs fréquents insectes - plantes suivants : 
- opilions et astéracées : support de 0.45
- Neuroptères et Apiacées : support de 0.39
- orthoptères et astéracées : support de 0.36
- hémiptères et astéracées : 0.35
- araignées et astéracées : 0.32
- Neuroptères et astéracées : 0.32
- lépidoptères et astéracées : 0.31
- coléoptères et astéracées : 0.3

- (?) siphonophores et apiacées: support de 0.40 (Les siphonophores  forment un ordre d'organismes zooplanctoniques gélatineux (?)...)

On complète le code ci-dessus afin de rechercher de potentielles règles d'associations importantes à partir de ces motifs fréquents. 

In [80]:
for insect in ['Hymenoptera', 'Diptera', 'Orthoptera', 'Coleoptera', 'Hemiptera', 'Araneae', 
               'Lepidoptera', 'Ericales', 'Neuroptera', 'Opiliones', 'Siphonophorae', 
               'Mecoptera', 'Dermaptera', 'Raphidioptera', 'Ephemeroptera', 'Blattodea']:

    data=df.where(df['insecte_ordre']==insect).dropna(subset=['insecte_ordre'])
    
    #Nettoyage des colonnes inutiles (on ne garde que plante_famille pour les taxons de plantes)
    to_drop= ['plante_genre', 'plante_espece', 'plante_sc',
       'plante_fr', 'plante_precision', 'grande_culture',
       'collection_date', 'collection_heure_debut', 
        'insecte_super_famille',
       'insecte_famille', 'insecte_sous_famille', 'insecte_genre',
       'insecte_espece', 'insecte_sc', 'insecte_fr',
       'insecte_denominationPlusPrecise', 'insecte_CdNomtaxref']
    data.drop(columns=to_drop, inplace=True)
    
    # One hot encoding
    df_encoded = pd.get_dummies(data)
    df_encoded = df_encoded.astype(bool)
    
    # Algo a priori sur le jeu encodé
    frequent_itemsets = apriori(df_encoded, min_support=0.3, use_colnames=True)

    # Filtrer les motifs fréquents pour une taille minimale de 2
    frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
    frequent_itemsets = frequent_itemsets[frequent_itemsets['length'] >= 2].drop(columns='length')

    # Classement par ordre décroissant de support 
    frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)
    #print(frequent_itemsets)
    #print(frequent_itemsets['itemsets'].unique())
    
    # Recherche de règles d'association avec un lift supérieur à 1 
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1, support_only=True)
    
    # Affichage des règles d'association intéressantes
    if len(rules)>0: 
        print(insect, " : ", rules)
    else : print(insect, ": no important rules")

Hymenoptera : no important rules
Diptera : no important rules
Orthoptera : no important rules
Coleoptera : no important rules
Hemiptera : no important rules
Araneae : no important rules
Lepidoptera : no important rules
Ericales : no important rules
Neuroptera : no important rules
Opiliones : no important rules
Siphonophorae : no important rules
Mecoptera : no important rules
Dermaptera : no important rules
Raphidioptera : no important rules
Ephemeroptera : no important rules
Blattodea : no important rules


Malgré les motifs fréquents relevés, aucune règle d'association importante n'a été soulevée (sur la base de la métrique du lift).

# Interprétations, conclusions et suites

Résumé des motifs fréquents relevés (support minimal de 0.3) :
- opilions et astéracées : support de 0.45. Les opilions se distinguent des araignées par l'absence de venin et de soie.  On les trouve souvent au sol, sur des arbustes, voire des arbres pour certaines espèces arboricoles, sous des objets divers (pierres, morceaux de bois), dans des morceaux de bois, dans les fissures des pierres, etc. 
    - Fréquents avec astéracées surement due à la sur-représentation des astéracées dans le jeu de données. 
- Neuroptères et Apiacées : support de 0.39
    - Les Neuroptères sont des insectes carnivores et réputés pour s'attaquer aux proies tendres. Certains d'entre eux comme les chrysopes sont des auxiliaires prédateurs efficaces, en particulier contre les pucerons. Or, les plantes de la famille des Apiacées, qui comprennent des espèces telles que le persil, le céleri et la carotte, sont connues pour être la cible de pucerons. Ainsi, la présence fréquente des Neuroptères près de plantes de la famille des Apiacées pourrait indiquer une relation bénéfique, telle que la prédation contre les pucerons. 
- orthoptères et astéracées : support de 0.36. Ces animaux se caractérisent par des ailes alignées par rapport au corps. On estime à 22 000 le nombre d'espèces présentes sur la planète. La grande majorité de ces espèces sont phytophages (se nourrissent de végétaux), bien que plusieurs soient régulièrement prédatrices. Cet ordre est scindé en deux sous-ordres : les ensifères (grillons et sauterelles) et les caelifères.(criquets). Les criquets ont un grand appétit pour divers types de végétation. Ils consomment une large gamme de plantes, notamment des cultures, des graminées, des feuilles, des fleurs, des fruits, des graines, de l’écorce et des céréales. Leur régime ne se limite pas à un type spécifique de plante mais englobe une sélection diversifiée.(scalecompanions.com) 
    - Fréquents avec astéracées surement due à la sur-représentation des astéracées dans le jeu de données.
- Hémiptères et astéracées : 0.35
- araignées et astéracées : 0.32
- Neuroptères et astéracées : 0.32
- lépidoptères et astéracées : 0.31. Forme adulte (ou imago) est communément appelée papillon.
- coléoptères et astéracées : 0.3. Certains sont détritivores, décomposant les débris de végétaux. D'autres se nourrissent de charogne ou d'excréments. D'autres encore se nourrissent de champignons. Certains sont phytophages; spécialistes ou généralistes, ils s'alimentent de pollen, de fleurs et de fruits.